In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/nifty50-stock-market-data/SBIN.csv',date_parser=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train=df[df['Date']<'2017-01-01']


In [ ]:
test=df[df['Date']>='2017-01-01']

In [ ]:
cols=['Open','High','Low','Close','Volume']
train=train[cols]
test=test[cols]

In [ ]:
scaler=MinMaxScaler()

nTrain=scaler.fit_transform(train)
nTrain

In [ ]:
xTrain=[]
yTrain=[]

for i in range(nTrain.shape[0]):
    temp=list()
    for j in range(5):
        if(j==3):
            yTrain.append(nTrain[i][j])
        else:
            temp.append(nTrain[i][j])
    xTrain.append(temp)





In [ ]:
xTrain,yTrain=np.array(xTrain),np.array(yTrain)

In [ ]:
xTrain.shape,yTrain.shape

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
model=Sequential()

model.add(Dense(units=50, activation='tanh', input_dim=4))
model.add(Dropout(0.2))

model.add(Dense(units=60, activation='tanh'))
model.add(Dropout(0.3))

model.add(Dense(units=80, activation='tanh'))
model.add(Dropout(0.4))

model.add(Dense(units=120, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
History = model.fit(xTrain,yTrain,epochs=100)

In [ ]:
plt.plot(History.history['loss'],color="r",label="Training Accuracy")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.title("Loss Function")
plt.show()

In [ ]:

nTest=scaler.transform(test)

In [ ]:
xTest=[]
yTest=[]

for i in range(nTest.shape[0]):
    temp=list()
    for j in range(5):
        if(j==3):
            yTest.append(nTest[i][j])
        else:
            temp.append(nTest[i][j])
    xTest.append(temp)

xTest,yTest=np.array(xTest),np.array(yTest)

In [ ]:
yPred=model.predict(xTest)

In [ ]:
yPred

In [ ]:
scaler.scale_,scaler.data_min_

In [ ]:
scaleVal=1/2.97428729e-04

In [ ]:
yPred=(yPred*scaleVal)+yTest.min()
yTest=(yTest*scaleVal)+yTest.min()

In [ ]:
yPred = (yPred*yTest.sum())/(yPred.sum())

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(yTest,color='red',label='Original')
plt.plot(yPred,color='blue',label='Predicted')

plt.title('Prediction')
plt.xlabel('Days')
plt.ylabel('Price')

plt.legend()
plt.show()
